## List 1

In [16]:
import numpy as np
import polars as pl
import scipy

Load the GEFCOM dataset

In [17]:
df = pl.read_csv('data/gefcom.txt', separator='\t', has_header=True)
df

Date,Hour,Price,SystemLoad,Load,Weekday,
f64,f64,f64,f64,f64,f64,str
2.0110101e7,0.0,43.17,15187.0,5091.0,6.0,null
2.0110101e7,1.0,36.24,14464.0,4918.0,6.0,null
2.0110101e7,2.0,34.64,13940.0,4763.0,6.0,null
2.0110101e7,3.0,33.76,13609.0,4660.0,6.0,null
2.0110101e7,4.0,33.08,13391.0,4599.0,6.0,null
…,…,…,…,…,…,…
2.0131217e7,19.0,113.92,23091.0,7167.0,2.0,null
2.0131217e7,20.0,107.26,22504.0,6958.0,2.0,null
2.0131217e7,21.0,89.02,21538.0,6707.0,2.0,null


Recast columns to appropriate types and drop the last column that has empty name ("")

In [18]:
df = df.with_columns(
    pl.col("Hour").cast(pl.UInt8),
    pl.col("Weekday").cast(pl.UInt8),
).drop(pl.col(""))
df

Date,Hour,Price,SystemLoad,Load,Weekday
f64,u8,f64,f64,f64,u8
2.0110101e7,0,43.17,15187.0,5091.0,6
2.0110101e7,1,36.24,14464.0,4918.0,6
2.0110101e7,2,34.64,13940.0,4763.0,6
2.0110101e7,3,33.76,13609.0,4660.0,6
2.0110101e7,4,33.08,13391.0,4599.0,6
…,…,…,…,…,…
2.0131217e7,19,113.92,23091.0,7167.0,2
2.0131217e7,20,107.26,22504.0,6958.0,2
2.0131217e7,21,89.02,21538.0,6707.0,2


### Task 1
Forecast daily average price using:
- naive one-step ahead forecasts,
- simple ETS,
- Holt-Winters (HW).

Use the first 360 days for calibration of the rest for testing.

Create a dataframe with the time series of daily average price

In [19]:
avg = df.group_by("Date").agg(pl.col("Price").mean().alias("Daily Average Price"), pl.col("Weekday").first())
avg = avg.sort("Date")
avg

Date,Daily Average Price,Weekday
f64,f64,u8
2.0110101e7,43.621667,6
2.0110102e7,43.015417,7
2.0110103e7,52.089583,1
2.0110104e7,51.254583,2
2.0110105e7,56.229167,3
…,…,…
2.0131213e7,145.694583,5
2.0131214e7,93.030417,6
2.0131215e7,77.7175,7


Calculate the naive one-step ahead forecasts

In [20]:
avg = avg.with_columns(Naive = pl.col("Daily Average Price").shift(1))

Calculate the simple ETS

In [21]:
T = 360
series = avg.select(pl.col("Daily Average Price")).to_numpy().ravel()
train, test = series[:T], series[T:]

In [22]:
def ets(params, obs):
    '''
    params:  parameters of the ETS model (alpha = params[0])
    obs:    'vector' of observations, which indices correspond to consecutive timesteps
    '''
    
    alpha = params[0]
    forecast = np.full(len(obs), np.nan) 
    forecast[1] = obs[0] # start with naive forecast

    for t in range(1, len(obs)-1):
        forecast[t+1] = alpha*obs[t] + (1-alpha)*forecast[t]
    
    return forecast

def loss_function_ets(params, obs):
    forecasts = ets(params, obs)
    return np.mean(np.square(forecasts[1:]-obs[1:]))

opt_params_ets = scipy.optimize.minimize(loss_function_ets, [0.5], args=(train), bounds = [(0, 1)]).x

Implement Holt-Winters:
- fill `...` in the code below
- optimize `params` to minimize MSE on the first 360 days of the data (training set) using `scipy.optimize.minimize`

In [23]:
def holtwinters(params, s, obs):
    """
    Calculates forecasts using Holt-Winters exponential smoothing.

    Args:
        params (tuple): parameters in the form of a 3-tuple (alpha, beta, gamma)
        s (int): seasonality period
        obs (1d numpy array): observed values of the timeseries to forecast
    
    Returns:
        numpy array of forecasts with the length equal to obs.
    """
    alpha, beta, gamma = params
    level = np.full(len(obs), np.nan)
    trend = np.full(len(obs), np.nan)
    season = np.full(len(obs), np.nan)
    forecast = np.full(len(obs), np.nan)

    # set initial values
    level[s-1] = np.mean(obs[:s])
    trend[s-1] = (np.mean(obs[s:2*s]) - np.mean(obs[:s]))/s
    season[:s] = obs[:s] - level[s-1]

    # iteratively compute consecutive forecasts
    for t in range(s, len(obs)-1):
        level[t] = alpha*(obs[t] - season[t-s]) + (1-alpha)*(level[t-1] + trend[t-1])
        trend[t] = beta*(level[t] - level[t-1]) + (1-beta)*trend[t-1]
        season[t] = gamma*(obs[t] - level[t]) + (1-gamma)*season[t-s]
        forecast[t+1] = level[t] + trend[t] + season[t-s+1]

    return forecast

def loss_function_hw(params, s, obs):    
    forecasts = holtwinters(params, s, obs)
    return np.mean(np.square(forecasts[2*s:]-obs[2*s:]))

opt_params_hw = scipy.optimize.minimize(loss_function_hw, [0.5, 0.5, 0.5], args=(7, train), bounds = [(0, 1), (0, 1), (0, 1)]).x

Calculate MAE and MSE of naive, ETS and Holt-Winters forecasts on the testing set

In [24]:
avg = avg.with_columns(ETS = ets(opt_params_ets, avg.select(pl.col("Daily Average Price")).to_numpy().ravel()))
avg = avg.with_columns(HW = holtwinters(opt_params_hw, 7, avg.select(pl.col("Daily Average Price")).to_numpy().ravel()))

In [25]:
for model in ["Naive", "ETS", "HW"]:
    errors = avg.select(pl.col(model)).to_numpy().ravel()[T:] - test
    mae = np.mean(np.abs(errors))
    rmse = np.sqrt(np.mean(np.square(errors)))
    print(f"{model} ::: MAE = {mae} ::: RMSE = {rmse}")

Naive ::: MAE = 6.0016314635272385 ::: RMSE = 11.1591860484464
ETS ::: MAE = 6.0016314635272385 ::: RMSE = 11.1591860484464
HW ::: MAE = 5.645741571187133 ::: RMSE = 10.965120636976996


### Task 2
Repeat task 1 using the first 720 days for calibration for ETS and HW models. Does the longer calibration window lead to more or less accurate forecasts? Compare the predictions only over the same out-of-sample period, i.e., days #721, #722, ... → why?

In [26]:
T = 720
series = avg.select(pl.col("Daily Average Price")).to_numpy().ravel()
train, test = series[:T], series[T:]

opt_params_ets = scipy.optimize.minimize(loss_function_ets, [0.5], args=(train), bounds = [(0, 1)]).x
opt_params_hw = scipy.optimize.minimize(loss_function_hw, [0.5, 0.5, 0.5], args=(7, train), bounds = [(0, 1), (0, 1), (0, 1)]).x

avg = avg.with_columns(ETS720 = ets(opt_params_ets, avg.select(pl.col("Daily Average Price")).to_numpy().ravel()))
avg = avg.with_columns(HW720 = holtwinters(opt_params_hw, 7, avg.select(pl.col("Daily Average Price")).to_numpy().ravel()))

In [27]:
for model in ["Naive", "ETS", "HW", "ETS720", "HW720"]:
    errors = avg.select(pl.col(model)).to_numpy().ravel()[T:] - test
    mae = np.mean(np.abs(errors))
    rmse = np.sqrt(np.mean(np.square(errors)))
    print(f"{model} ::: MAE = {mae} ::: RMSE = {rmse}")

Naive ::: MAE = 7.221056629834254 ::: RMSE = 12.9436539533331
ETS ::: MAE = 7.221056629834254 ::: RMSE = 12.9436539533331
HW ::: MAE = 6.828768540018034 ::: RMSE = 12.679194280271247
ETS720 ::: MAE = 7.221056629834254 ::: RMSE = 12.9436539533331
HW720 ::: MAE = 6.828911519208887 ::: RMSE = 12.625192735510538


### Task 3
Repeat tasks 1 and 2 but forecast all hours of the day, treat prices at each hour of the day as separate time series.

In [28]:
df = df.with_columns(
    Naive = pl.col("Price").shift(24),
    ETS = pl.lit(None),
    HW = pl.lit(None),
    ETS720 = pl.lit(None),
    HW720 = pl.lit(None),
    )

for h in range(0, 24):
    hourly = df.filter(pl.col('Hour') == h).sort('Date')
    T = 360
    train = hourly.select(pl.col("Price")).to_numpy().ravel()[:T]
    opt_params_ets = scipy.optimize.minimize(loss_function_ets, [0.5], args=(train), bounds = [(0, 1)]).x
    opt_params_hw = scipy.optimize.minimize(loss_function_hw, [0.5, 0.5, 0.5], args=(7, train), bounds = [(0, 1), (0, 1), (0, 1)]).x

    hourly = hourly.with_columns(
        ETS_h = ets(opt_params_ets, hourly.select(pl.col("Price")).to_numpy().ravel()),
        HW_h =  holtwinters(opt_params_hw, 7, hourly.select(pl.col("Price")).to_numpy().ravel())
    )

    T = 720
    train = hourly.select(pl.col("Price")).to_numpy().ravel()[:T]
    opt_params_ets = scipy.optimize.minimize(loss_function_ets, [0.5], args=(train), bounds = [(0, 1)]).x
    opt_params_hw = scipy.optimize.minimize(loss_function_hw, [0.5, 0.5, 0.5], args=(7, train), bounds = [(0, 1), (0, 1), (0, 1)]).x

    hourly = hourly.with_columns(
        ETS720_h = ets(opt_params_ets, hourly.select(pl.col("Price")).to_numpy().ravel()),
        HW720_h =  holtwinters(opt_params_hw, 7, hourly.select(pl.col("Price")).to_numpy().ravel())
    )
    
    df = df.join(hourly.select(pl.col(["Date", "Hour", "ETS_h", "HW_h", "ETS720_h", "HW720_h"])), on=["Date", "Hour"], how="left")

    for colname in ["ETS", "HW", "ETS720", "HW720"]:
        df = df.with_columns(
            pl.when(pl.col('Hour') == h)
                .then(pl.col(f"{colname}_h"))
                .otherwise(pl.col(colname))
                .alias(colname)
        )
        df = df.drop(f"{colname}_h")

In [29]:
T = 720*24
test = df.select(pl.col("Price")).to_numpy().ravel()[T:]
for model in ["Naive", "ETS", "HW", "ETS720", "HW720"]:
    errors = df.select(pl.col(model)).to_numpy().ravel()[T:] - test
    mae = np.mean(np.abs(errors))
    rmse = np.sqrt(np.mean(np.square(errors)))
    print(f"{model} ::: MAE = {mae} ::: RMSE = {rmse}")

Naive ::: MAE = 8.191443370165747 ::: RMSE = 15.633654245686285
ETS ::: MAE = 8.301894551224693 ::: RMSE = 15.79701287682037
HW ::: MAE = 7.892520204090258 ::: RMSE = 15.28609169889551
ETS720 ::: MAE = 8.299308834668821 ::: RMSE = 15.813918802554884
HW720 ::: MAE = 7.965533674487849 ::: RMSE = 15.400091416261827


**Homework** 

Modify the codes so that the most recent data is used for estimating the model with shorter training window (360 days).